# Import librairies

In [1]:
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import string
sys.path.append("../utile")
from utile import config_parse
from selenium.webdriver.common.by import By

# Define driver

In [2]:
config = config_parse()

In [3]:
login_linkedin = config["login_linkedin"]
password_linkedin = config["password_linkedin"]

login_insta = config["login_insta"]
password_insta = config["password_insta"]

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(chrome_options=chrome_options)


/tmp/ipykernel_2333/3486248830.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [5]:
# Connect to Linkedin
try:
    driver.get("https://www.linkedin.com")
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH,'/html/body/nav/div/a[2]').click()
    driver.find_element(By.ID,'username').send_keys(login_linkedin)
    driver.find_element(By.ID,'password').send_keys(password_linkedin)
    driver.implicitly_wait(1)
    driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()
except:
    print("Failed to connect")
    pass



In [6]:
# Connect to instagram
try:
    driver.get("https://www.instagram.com")
    driver.implicitly_wait(3)
    try:
        driver.find_element(By.XPATH,'/html/body/div[4]/div/div/button[2]').click()
    except:
        print("No need to accept cookies")
        pass
    driver.find_element(By.NAME,'username').send_keys(login_linkedin)
    driver.find_element(By.NAME,'password').send_keys(password_insta)
    driver.implicitly_wait(1)
    driver.find_element(By.XPATH,'//*[@id="loginForm"]/div/div[3]/button').click()

except:
    print("Failed to connect")
    pass


In [7]:
df = pd.read_csv("../data/escp_training_set.csv")

In [54]:
# Get all followers of linkedin for n companies (to improve)
follow_linkedin = []
publications_insta = []
for i,company in enumerate (df["company_name"].to_list()[:200]):
    #if i%10==0:
    #    print(i)
    query = company + " linkedin"
    driver.get("https://www.google.co.uk/search?q=" + query)
    try:
        driver.find_element(By.XPATH,'//*[@id="L2AGLb"]/div').click()
    except:
        pass
    soup = BeautifulSoup(driver.page_source)
    try:
        driver.find_element(by=By.TAG_NAME, value='h3').click()
        try:
            nb_followers_linkedin = int(''.join(driver.find_elements(by=By.CLASS_NAME, value='org-top-card-summary-info-list__info-item')[-1].text.replace(" abonnés","").split()))
            follow_linkedin.append(nb_followers_linkedin)
        except:
            try:
                driver.find_element(by=By.TAG_NAME, value='h3').click()
                nb_followers_linkedin = int(''.join(driver.find_elements(by=By.CLASS_NAME, value='org-top-card-summary-info-list__info-item')[-1].text.replace(" abonnés","").split()))
                follow_linkedin.append(nb_followers_linkedin)
            except:
                follow_linkedin.append(0)
    except:
        follow_linkedin.append(0)
        


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [57]:
followers = pd.DataFrame(follow_linkedin, columns=["Followers"])

In [58]:
followers.value_counts()

Followers
0            44
1092          2
8828          1
6238          1
6630          1
             ..
981           1
986           1
1037          1
1050          1
2794803       1
Length: 156, dtype: int64

In [59]:
followers.to_csv("followers.csv")

In [57]:
#Information
print(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[2].text)
#Summary
print(driver.find_elements_by_class_name('overflow-hidden')[0].text.replace("\n", " "))

10 634 abonnés



/tmp/ipykernel_808/1248601977.py:2: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  print(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[2].text)
/tmp/ipykernel_808/1248601977.py:4: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  print(driver.find_elements_by_class_name('overflow-hidden')[0].text.replace("\n", " "))


# Test

In [64]:
# Get info for instagram
company = "Amano"
query = company + " instagram"
driver.get("https://www.google.co.uk/search?q=" + query)
try:
    driver.find_element(By.XPATH,'//*[@id="L2AGLb"]/div').click()
except:
    pass
soup = BeautifulSoup(driver.page_source)
driver.find_element_by_tag_name('h3').click()

#driver.get(soup.find_all('a')[20]["href"])

/tmp/ipykernel_808/1829854153.py:9: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('h3').click()


In [71]:
# See number of publications
driver.find_elements_by_class_name('wW3k-')[0].text.replace("\n", " ").split(" publications")[0].split(" ")[-1]

/tmp/ipykernel_808/3283888356.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  driver.find_elements_by_class_name('wW3k-')[0].text.replace("\n", " ").split(" publications")[0].split(" ")[-1]


'437'

In [69]:
# See number of followers
driver.find_elements_by_class_name('wW3k-')[0].text.replace("\n", " ").split(" abonnés")[0].split(" ")[-1]

/tmp/ipykernel_808/3066775476.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  driver.find_elements_by_class_name('wW3k-')[0].text.replace("\n", " ").split(" abonnés")[0].split(" ")[-1]


'19,6K'

In [21]:
company = "Aurizon"
query = company + " linkedin"
driver.get("https://www.google.co.uk/search?q=" + query)
try:
    driver.find_element(By.XPATH,'//*[@id="L2AGLb"]/div').click()
except:
    pass
soup = BeautifulSoup(driver.page_source)
try:
    driver.find_element_by_tag_name('h3').click()
    try:
        print(driver.current_url)
        print(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[4].text)
        nb_followers_linkedin = int(''.join(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[-1].text.replace(" abonnés","").split()))
    except:
        print(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[-1].text)
        driver.find_element_by_tag_name('h3').click()
        nb_followers_linkedin = int(''.join(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[-1].text.replace(" abonnés","").split()))
        print(driver.current_url)


except:
    pass

/tmp/ipykernel_2333/2191086222.py:10: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('h3').click()
/tmp/ipykernel_2333/2191086222.py:13: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  print(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[4].text)


https://www.google.co.uk/search?q=Aurizon%20linkedin
45 712 abonnés
https://www.linkedin.com/company/aurizon/?originalSubdomain=au


/tmp/ipykernel_2333/2191086222.py:16: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  print(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[-1].text)
/tmp/ipykernel_2333/2191086222.py:17: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('h3').click()
/tmp/ipykernel_2333/2191086222.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  nb_followers_linkedin = int(''.join(driver.find_elements_by_class_name('org-top-card-summary-info-list__info-item')[-1].text.replace(" abonnés","").split()))


In [113]:
# About us section of linkedin to get description
soup = BeautifulSoup(driver.page_source,"html.parser")
soup.find_all(class_="about-us__description")[0].text

IndexError: list index out of range